In [1]:
import numpy as np #importing numpy
import pandas as pd #importing pandas
import PIL #importing pillow for image processing
from PIL import Image#importing image from pillow library
import skimage#importing skimage library
import os#importing os library
import time#importing time library

#Capture start time of the program
start_time = time.time()

In [2]:
from string import digits
#code to remove numbers in a string and assigning it to the variable
r_digits = str.maketrans('', '', digits)
#taking source data folder path to a varible
rawdata_path = 'D://DKIT//dissertation//Dataset_of_Tomato_Leaves//plantvillage//Preprocessed_data//64//no_bias//'
#changing current librbay to source data
os.chdir(rawdata_path)
#taking sub-directories to a list
dirs = os.listdir()
directories = []
diseases = []
#Taking directories into a list and extract disease name from folder name
for x in dirs:
    res = x.translate(r_digits).lower()
    diseases.append(res)
    temp = rawdata_path + x + '//'
    directories.append(temp)

In [3]:
for i in directories:
    os.chdir(i)
    pics = os.listdir()
    #reading pics one by one in each directory
    for j in pics:
        image = Image.open(j,mode='r')
        size = (64,64)
        image = image.resize(size)
        image = image.save(j)

In [4]:
#creating different dataframes for saving data in different vector forms
#gray data
gray_df = pd.DataFrame(columns=['mean','disease'])
#rgb data
rgb_df = pd.DataFrame(columns=['mean','disease'])
#red data
red_df = pd.DataFrame(columns=['mean','disease'])
#green data
green_df = pd.DataFrame(columns=['mean','disease'])
#blue data
blue_df = pd.DataFrame(columns=['mean','disease'])

In [5]:
#extracting data from images
x = 0
#creating enpty directories to save the vector from in to a list
gray_test = []
rgb_final = []
red_final = []
green_final = []
blue_final = []
disease = []
#reading data directory by directory
for i in directories:
    os.chdir(i)
    k = diseases[x]
    pics = os.listdir()
    #reading pics one by one in each directory
    for j in pics:
        temp_list = []
        rgb_temp = []
        red_temp = []
        green_temp = []
        blue_temp = []
        gray = []
        data = []
        red = []
        green = []
        blue = []
        #opening the image in rgb format
        image = Image.open(j,mode='r')
        data = image.convert('RGB')
        data = np.array(data)
        #extracting only red components
        red = data[:,:,0]
        #extracting only green components
        green = data[:,:,1]
        #extracting only blue components
        blue = data[:,:,2]
        data = data.ravel()
        #saving rgb vector data into list
        rgb_final.append(data)
        rgb_temp.append(np.mean(data))
        rgb_temp.append(k)
        rgb_df.loc[len(rgb_df)] = rgb_temp
        #saving red channel data into dataframe and vector data to a list.
        red = red.ravel()
        red_final.append(red)
        red_temp.append(np.mean(red))
        red_temp.append(k)
        red_df.loc[len(red_df)] = red_temp
        #saving blue channel data into dataframe and vector data to a list.
        blue = blue.ravel()
        blue_final.append(blue)
        blue_temp.append(np.mean(blue))
        blue_temp.append(k)
        blue_df.loc[len(blue_df)] = blue_temp
        #saving green channel data into dataframe and vector data to a list.
        green = green.ravel()
        green_final.append(green)
        green_temp.append(np.mean(green))
        green_temp.append(k)
        green_df.loc[len(green_df)] = green_temp
        #opening the image in gray format
        gray = image.convert('L')
        gray = np.array(gray)
        gray = gray.ravel()
        gray_test.append(gray)
        temp_list.append(np.mean(gray))
        temp_list.append(k)       
        gray_df.loc[len(gray_df)] = temp_list
    x = x+1

In [6]:
#changing path
os.chdir('D://DKIT//dissertation//Dataset_of_Tomato_Leaves//plantvillage//Preprocessed_data//64//no_bias_vector//')

In [7]:
#saving the lists as numpy arrays in the local system
np.save('gray_test',gray_test)
np.save('rgb_data',rgb_final)
np.save('red_data',red_final)
np.save('green_data',green_final)
np.save('blue_data',blue_final)

In [8]:
#saving the dataframe as csv files in the local system
gray_df.to_csv('gray_data.csv',index=False)
rgb_df.to_csv('rgb_data.csv',index=False)
red_df.to_csv('red_data.csv',index=False)
green_df.to_csv('green_data.csv',index=False)
blue_df.to_csv('blue_data.csv',index=False)

# KNN on rgb data

In [9]:
#reading numpy array data
rgb_test = np.load('rgb_data.npy')
#reading csv file
rgb_df = pd.read_csv('rgb_data.csv')

In [10]:
loaded = []
for i in rgb_test:
    k = []
    k = np.asarray(i,dtype='uint8')
    loaded.append(k)

In [11]:
#assigning features and target
features = loaded
target = rgb_df['disease']
#splitting the data to 80 percent and 20 percent
from sklearn.model_selection import train_test_split
Ftrain,Ftest,Ltrain,Ltest=train_test_split(features,target, test_size=0.20, random_state=24)
#scaling the data using standard scaler
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
Ftrain = scaler.fit_transform(Ftrain)
Ftest = scaler.fit_transform(Ftest)

In [12]:
#K-Model on the data
from sklearn.neighbors import KNeighborsClassifier
kmodel = KNeighborsClassifier(n_neighbors=75)

In [13]:
#k-model fitting and accuracy
kmodel.fit(Ftrain,Ltrain)
print(kmodel.score(Ftest,Ltest))

0.4324561403508772


In [14]:
#importing classification report and confusion matrix
from sklearn.metrics import classification_report, confusion_matrix

In [15]:
#printing results of the kmodel
y_predict = kmodel.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.25      0.78      0.37       115
                 early_blight       1.00      0.01      0.02       120
                      healthy       0.38      0.81      0.52       125
                  late_blight       0.41      0.20      0.27       125
                    leaf_mold       0.76      0.57      0.65       127
           septoria_leaf_spot       0.68      0.32      0.44       117
                  target_spot       0.56      0.29      0.38       115
          tomato_mosaic_virus       0.62      0.39      0.48        54
tomato_yellow_leaf_curl_virus       0.84      0.37      0.51       114
      two-spotted_spider_mite       0.46      0.54      0.50       128

                     accuracy                           0.43      1140
                    macro avg       0.60      0.43      0.41      1140
                 weighted avg       0.59      0.43      0.41      1140

[[ 

# Logistic on rgb data

In [16]:
#importing logistic data
from sklearn.linear_model import LogisticRegression
model=LogisticRegression(solver='lbfgs', max_iter=10000)
model.fit(Ftrain,Ltrain)
print(model.score(Ftest,Ltest))

0.6640350877192982


In [17]:
#classification report
y_predict = model.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.64      0.72      0.68       115
                 early_blight       0.50      0.42      0.45       120
                      healthy       0.75      0.62      0.68       125
                  late_blight       0.59      0.58      0.59       125
                    leaf_mold       0.74      0.73      0.74       127
           septoria_leaf_spot       0.61      0.64      0.63       117
                  target_spot       0.63      0.73      0.67       115
          tomato_mosaic_virus       0.79      0.85      0.82        54
tomato_yellow_leaf_curl_virus       0.80      0.82      0.81       114
      two-spotted_spider_mite       0.65      0.64      0.64       128

                     accuracy                           0.66      1140
                    macro avg       0.67      0.68      0.67      1140
                 weighted avg       0.66      0.66      0.66      1140

[[8

# SVM(linear) on RGB data

In [18]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

clf = SVC(kernel='linear')
clf.fit(Ftrain,Ltrain)
y_pred = clf.predict(Ftest)
print(accuracy_score(Ltest,y_pred))

0.7175438596491228


In [19]:
#classification report
y_predict = clf.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.65      0.81      0.72       115
                 early_blight       0.56      0.54      0.55       120
                      healthy       0.87      0.83      0.85       125
                  late_blight       0.66      0.64      0.65       125
                    leaf_mold       0.76      0.75      0.75       127
           septoria_leaf_spot       0.70      0.63      0.66       117
                  target_spot       0.66      0.75      0.70       115
          tomato_mosaic_virus       0.84      0.91      0.88        54
tomato_yellow_leaf_curl_virus       0.85      0.83      0.84       114
      two-spotted_spider_mite       0.72      0.60      0.66       128

                     accuracy                           0.72      1140
                    macro avg       0.73      0.73      0.73      1140
                 weighted avg       0.72      0.72      0.72      1140

[[ 

# SVM(poly) on RGB data

In [20]:
svm_poly = SVC(kernel='poly')
svm_poly.fit(Ftrain,Ltrain)
y_pred = svm_poly.predict(Ftest)
print(accuracy_score(Ltest,y_pred))

0.6710526315789473


In [21]:
#classification report
y_predict = svm_poly.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.46      0.85      0.60       115
                 early_blight       0.78      0.29      0.42       120
                      healthy       0.62      0.96      0.75       125
                  late_blight       0.81      0.48      0.60       125
                    leaf_mold       0.81      0.79      0.80       127
           septoria_leaf_spot       0.73      0.50      0.60       117
                  target_spot       0.64      0.75      0.69       115
          tomato_mosaic_virus       0.81      0.46      0.59        54
tomato_yellow_leaf_curl_virus       0.83      0.80      0.82       114
      two-spotted_spider_mite       0.68      0.71      0.70       128

                     accuracy                           0.67      1140
                    macro avg       0.72      0.66      0.66      1140
                 weighted avg       0.71      0.67      0.66      1140

[[ 

# SVM(RBF) on RGB data

In [22]:
svm_rbf = SVC(kernel='rbf')
svm_rbf.fit(Ftrain,Ltrain)
y_pred = svm_rbf.predict(Ftest)
print(accuracy_score(Ltest,y_pred))

0.769298245614035


In [23]:
#classification report
y_predict = svm_rbf.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.77      0.86      0.81       115
                 early_blight       0.54      0.57      0.56       120
                      healthy       0.93      0.92      0.93       125
                  late_blight       0.76      0.75      0.76       125
                    leaf_mold       0.86      0.78      0.82       127
           septoria_leaf_spot       0.76      0.65      0.70       117
                  target_spot       0.71      0.71      0.71       115
          tomato_mosaic_virus       0.78      0.83      0.80        54
tomato_yellow_leaf_curl_virus       0.83      0.87      0.85       114
      two-spotted_spider_mite       0.76      0.77      0.76       128

                     accuracy                           0.77      1140
                    macro avg       0.77      0.77      0.77      1140
                 weighted avg       0.77      0.77      0.77      1140

[[ 

# SVM(sigmoid) on RGB data

In [24]:
svm_si = SVC(kernel='sigmoid')
svm_si.fit(Ftrain,Ltrain)
y_pred = svm_si.predict(Ftest)
print(accuracy_score(Ltest,y_pred))

0.5


In [25]:
#classification report
y_predict = svm_si.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.49      0.58      0.53       115
                 early_blight       0.45      0.31      0.36       120
                      healthy       0.71      0.67      0.69       125
                  late_blight       0.36      0.47      0.41       125
                    leaf_mold       0.74      0.47      0.58       127
           septoria_leaf_spot       0.24      0.47      0.32       117
                  target_spot       0.51      0.45      0.48       115
          tomato_mosaic_virus       0.67      0.57      0.62        54
tomato_yellow_leaf_curl_virus       0.83      0.50      0.62       114
      two-spotted_spider_mite       0.61      0.53      0.57       128

                     accuracy                           0.50      1140
                    macro avg       0.56      0.50      0.52      1140
                 weighted avg       0.55      0.50      0.51      1140

[[6

# Random Forest on RGB 

In [26]:
from sklearn.ensemble import RandomForestClassifier

In [27]:
classifier_rf = RandomForestClassifier(random_state=42, n_jobs=-1, max_depth=5,n_estimators=100, oob_score=True)

In [28]:
classifier_rf.fit(Ftrain, Ltrain)

RandomForestClassifier(max_depth=5, n_jobs=-1, oob_score=True, random_state=42)

In [29]:
classifier_rf.oob_score_

0.5409080938802369

In [30]:
y_predict = classifier_rf.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.44      0.66      0.53       115
                 early_blight       0.71      0.10      0.18       120
                      healthy       0.66      0.86      0.74       125
                  late_blight       0.66      0.38      0.48       125
                    leaf_mold       0.64      0.69      0.66       127
           septoria_leaf_spot       0.57      0.48      0.52       117
                  target_spot       0.44      0.70      0.54       115
          tomato_mosaic_virus       0.50      0.02      0.04        54
tomato_yellow_leaf_curl_virus       0.52      0.84      0.64       114
      two-spotted_spider_mite       0.50      0.45      0.48       128

                     accuracy                           0.54      1140
                    macro avg       0.56      0.52      0.48      1140
                 weighted avg       0.57      0.54      0.51      1140

[[ 

# Gaussian Naive Bayes on RGB data

In [31]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb.fit(Ftrain, Ltrain)

GaussianNB()

In [32]:
y_pred = gnb.predict(Ftest)

In [33]:
from sklearn import metrics
metrics.accuracy_score(Ltest, y_pred)

0.45614035087719296

In [34]:
class_names = target.unique()
print(classification_report(Ltest, y_pred, target_names=class_names))
print(confusion_matrix(Ltest, y_pred))

                               precision    recall  f1-score   support

               bacterial_spot       0.47      0.69      0.56       115
                 early_blight       0.37      0.31      0.34       120
                      healthy       0.44      0.61      0.51       125
                  late_blight       0.34      0.24      0.28       125
                    leaf_mold       0.73      0.43      0.54       127
           septoria_leaf_spot       0.41      0.41      0.41       117
                  target_spot       0.47      0.43      0.45       115
          tomato_mosaic_virus       0.34      0.69      0.45        54
tomato_yellow_leaf_curl_virus       0.65      0.45      0.53       114
      two-spotted_spider_mite       0.45      0.45      0.45       128

                     accuracy                           0.46      1140
                    macro avg       0.47      0.47      0.45      1140
                 weighted avg       0.47      0.46      0.45      1140

[[7

# Bernouli Naive bayes on RGB data

In [35]:
from sklearn.naive_bayes import BernoulliNB

In [36]:
bnb = BernoulliNB()
bnb.fit(Ftrain, Ltrain)

BernoulliNB()

In [37]:
y_pred = bnb.predict(Ftest)

In [38]:
metrics.accuracy_score(Ltest, y_pred)

0.4087719298245614

In [39]:
class_names = target.unique()
print(classification_report(Ltest, y_pred, target_names=class_names))
print(confusion_matrix(Ltest, y_pred))

                               precision    recall  f1-score   support

               bacterial_spot       0.37      0.63      0.47       115
                 early_blight       0.34      0.22      0.27       120
                      healthy       0.41      0.55      0.47       125
                  late_blight       0.19      0.11      0.14       125
                    leaf_mold       0.70      0.39      0.50       127
           septoria_leaf_spot       0.34      0.51      0.41       117
                  target_spot       0.50      0.44      0.47       115
          tomato_mosaic_virus       0.31      0.48      0.38        54
tomato_yellow_leaf_curl_virus       0.65      0.43      0.52       114
      two-spotted_spider_mite       0.42      0.38      0.40       128

                     accuracy                           0.41      1140
                    macro avg       0.42      0.42      0.40      1140
                 weighted avg       0.43      0.41      0.40      1140

[[7

# Decision Tree on RGB data

In [40]:
from sklearn.tree import DecisionTreeClassifier
classifier_tree = DecisionTreeClassifier()

In [41]:
classifier_tree.fit(Ftrain, Ltrain)
classifier_tree.score(Ftest,Ltest)

0.39035087719298245

In [42]:
y_predict = classifier_tree.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.49      0.54      0.51       115
                 early_blight       0.31      0.33      0.32       120
                      healthy       0.50      0.42      0.45       125
                  late_blight       0.33      0.37      0.35       125
                    leaf_mold       0.49      0.46      0.48       127
           septoria_leaf_spot       0.41      0.42      0.42       117
                  target_spot       0.36      0.35      0.35       115
          tomato_mosaic_virus       0.11      0.11      0.11        54
tomato_yellow_leaf_curl_virus       0.49      0.50      0.50       114
      two-spotted_spider_mite       0.32      0.30      0.31       128

                     accuracy                           0.39      1140
                    macro avg       0.38      0.38      0.38      1140
                 weighted avg       0.40      0.39      0.39      1140

[[6

# KNN on grayscale data

In [43]:
#reading numpy array data
gray_test = np.load('gray_test.npy')
#reading csv file
gray_df = pd.read_csv('gray_data.csv')

In [44]:
loaded = []
for i in gray_test:
    k = []
    k = np.asarray(i,dtype='uint8')
    loaded.append(k)

In [45]:
features = loaded
target = gray_df['disease']
Ftrain,Ftest,Ltrain,Ltest=train_test_split(features,target, test_size=0.20, random_state=20)
Ftrain = scaler.fit_transform(Ftrain)
Ftest = scaler.fit_transform(Ftest)

In [46]:
kmodel.fit(Ftrain,Ltrain)
kmodel.score(Ftest,Ltest)

0.356140350877193

In [47]:
y_predict = kmodel.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.27      0.74      0.39       124
                 early_blight       0.00      0.00      0.00       114
                      healthy       0.36      0.69      0.47       131
                  late_blight       0.26      0.23      0.24       124
                    leaf_mold       0.48      0.50      0.49       117
           septoria_leaf_spot       0.44      0.42      0.43       114
                  target_spot       0.38      0.15      0.22       111
          tomato_mosaic_virus       0.26      0.12      0.16        60
tomato_yellow_leaf_curl_virus       0.45      0.12      0.18       121
      two-spotted_spider_mite       0.50      0.42      0.46       124

                     accuracy                           0.36      1140
                    macro avg       0.34      0.34      0.30      1140
                 weighted avg       0.34      0.36      0.32      1140

[[9

C:\Users\ravic\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ravic\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ravic\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Logistic on grayscale data

In [48]:
from sklearn.linear_model import LogisticRegression
model=LogisticRegression(solver='lbfgs', max_iter=10000)
model.fit(Ftrain,Ltrain)
model.score(Ftest,Ltest)

0.3157894736842105

In [49]:
y_predict = model.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.32      0.38      0.35       124
                 early_blight       0.24      0.28      0.26       114
                      healthy       0.49      0.44      0.46       131
                  late_blight       0.18      0.21      0.20       124
                    leaf_mold       0.38      0.38      0.38       117
           septoria_leaf_spot       0.30      0.30      0.30       114
                  target_spot       0.21      0.18      0.19       111
          tomato_mosaic_virus       0.28      0.13      0.18        60
tomato_yellow_leaf_curl_virus       0.27      0.25      0.26       121
      two-spotted_spider_mite       0.45      0.49      0.47       124

                     accuracy                           0.32      1140
                    macro avg       0.31      0.30      0.30      1140
                 weighted avg       0.32      0.32      0.31      1140

[[4

# SVM(linear) on Grayscale data

In [50]:
clf = SVC(kernel='linear')
clf.fit(Ftrain,Ltrain)
y_pred = clf.predict(Ftest)
print(accuracy_score(Ltest,y_pred))

0.3868421052631579


In [51]:
y_predict = clf.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.33      0.44      0.38       124
                 early_blight       0.29      0.35      0.32       114
                      healthy       0.53      0.53      0.53       131
                  late_blight       0.31      0.32      0.32       124
                    leaf_mold       0.44      0.46      0.45       117
           septoria_leaf_spot       0.39      0.35      0.37       114
                  target_spot       0.29      0.26      0.27       111
          tomato_mosaic_virus       0.41      0.23      0.30        60
tomato_yellow_leaf_curl_virus       0.38      0.32      0.35       121
      two-spotted_spider_mite       0.52      0.49      0.50       124

                     accuracy                           0.39      1140
                    macro avg       0.39      0.38      0.38      1140
                 weighted avg       0.39      0.39      0.39      1140

[[5

# SVM(poly) on grayscale data

In [52]:
svm_poly = SVC(kernel='poly')
svm_poly.fit(Ftrain,Ltrain)
y_pred = svm_poly.predict(Ftest)
print(accuracy_score(Ltest,y_pred))

0.5026315789473684


In [53]:
#classification report
y_predict = svm_poly.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.44      0.68      0.53       124
                 early_blight       0.54      0.19      0.28       114
                      healthy       0.48      0.86      0.62       131
                  late_blight       0.50      0.15      0.23       124
                    leaf_mold       0.50      0.65      0.56       117
           septoria_leaf_spot       0.68      0.42      0.52       114
                  target_spot       0.47      0.56      0.51       111
          tomato_mosaic_virus       0.60      0.05      0.09        60
tomato_yellow_leaf_curl_virus       0.42      0.57      0.48       121
      two-spotted_spider_mite       0.69      0.62      0.65       124

                     accuracy                           0.50      1140
                    macro avg       0.53      0.48      0.45      1140
                 weighted avg       0.53      0.50      0.47      1140

[[ 

# SVM(RBF) on grayscale data

In [54]:
svm_rbf = SVC(kernel='rbf')
svm_rbf.fit(Ftrain,Ltrain)
y_pred = svm_rbf.predict(Ftest)
print(accuracy_score(Ltest,y_pred))

0.6280701754385964


In [55]:
#classification report
y_predict = svm_rbf.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.75      0.65      0.70       124
                 early_blight       0.46      0.49      0.48       114
                      healthy       0.70      0.87      0.77       131
                  late_blight       0.61      0.54      0.58       124
                    leaf_mold       0.67      0.62      0.64       117
           septoria_leaf_spot       0.56      0.61      0.58       114
                  target_spot       0.55      0.63      0.59       111
          tomato_mosaic_virus       0.60      0.47      0.52        60
tomato_yellow_leaf_curl_virus       0.64      0.61      0.63       121
      two-spotted_spider_mite       0.72      0.69      0.70       124

                     accuracy                           0.63      1140
                    macro avg       0.63      0.62      0.62      1140
                 weighted avg       0.63      0.63      0.63      1140

[[ 

# SVM(sigmoid) on grayscale data

In [56]:
svm_si = SVC(kernel='sigmoid')
svm_si.fit(Ftrain,Ltrain)
y_pred = svm_si.predict(Ftest)
print(accuracy_score(Ltest,y_pred))

0.27807017543859647


In [57]:
#classification report
y_predict = svm_si.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.38      0.47      0.42       124
                 early_blight       0.18      0.20      0.19       114
                      healthy       0.35      0.22      0.27       131
                  late_blight       0.19      0.27      0.22       124
                    leaf_mold       0.57      0.22      0.32       117
           septoria_leaf_spot       0.17      0.37      0.23       114
                  target_spot       0.21      0.23      0.22       111
          tomato_mosaic_virus       0.46      0.10      0.16        60
tomato_yellow_leaf_curl_virus       0.34      0.23      0.27       121
      two-spotted_spider_mite       0.47      0.38      0.42       124

                     accuracy                           0.28      1140
                    macro avg       0.33      0.27      0.27      1140
                 weighted avg       0.33      0.28      0.28      1140

[[5

# Random Forest on Grayscale

In [58]:
classifier_rf.fit(Ftrain, Ltrain)

RandomForestClassifier(max_depth=5, n_jobs=-1, oob_score=True, random_state=42)

In [59]:
classifier_rf.oob_score_

0.442640930028515

In [60]:
y_predict = classifier_rf.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.46      0.61      0.53       124
                 early_blight       0.40      0.18      0.25       114
                      healthy       0.47      0.74      0.57       131
                  late_blight       0.48      0.28      0.36       124
                    leaf_mold       0.57      0.54      0.55       117
           septoria_leaf_spot       0.58      0.46      0.51       114
                  target_spot       0.38      0.67      0.48       111
          tomato_mosaic_virus       0.00      0.00      0.00        60
tomato_yellow_leaf_curl_virus       0.43      0.31      0.36       121
      two-spotted_spider_mite       0.48      0.60      0.54       124

                     accuracy                           0.47      1140
                    macro avg       0.42      0.44      0.42      1140
                 weighted avg       0.45      0.47      0.44      1140

[[7

C:\Users\ravic\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ravic\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ravic\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Gaussian Naive bayes on Grayscale

In [61]:
gnb.fit(Ftrain, Ltrain)

GaussianNB()

In [62]:
y_pred = gnb.predict(Ftest)
metrics.accuracy_score(Ltest, y_pred)

0.41578947368421054

In [63]:
class_names = target.unique()
print(classification_report(Ltest, y_pred, target_names=class_names))
print(confusion_matrix(Ltest, y_pred))

                               precision    recall  f1-score   support

               bacterial_spot       0.49      0.69      0.57       124
                 early_blight       0.38      0.37      0.37       114
                      healthy       0.45      0.63      0.52       131
                  late_blight       0.33      0.20      0.25       124
                    leaf_mold       0.61      0.30      0.40       117
           septoria_leaf_spot       0.40      0.47      0.43       114
                  target_spot       0.51      0.39      0.44       111
          tomato_mosaic_virus       0.29      0.60      0.39        60
tomato_yellow_leaf_curl_virus       0.30      0.13      0.18       121
      two-spotted_spider_mite       0.40      0.44      0.42       124

                     accuracy                           0.42      1140
                    macro avg       0.41      0.42      0.40      1140
                 weighted avg       0.42      0.42      0.40      1140

[[8

# Bernouli Naive bayes on grayscale data

In [64]:
bnb.fit(Ftrain, Ltrain)
y_pred = bnb.predict(Ftest)
metrics.accuracy_score(Ltest, y_pred)

0.34385964912280703

In [65]:
class_names = target.unique()
print(classification_report(Ltest, y_pred, target_names=class_names))
print(confusion_matrix(Ltest, y_pred))

                               precision    recall  f1-score   support

               bacterial_spot       0.40      0.69      0.51       124
                 early_blight       0.27      0.18      0.21       114
                      healthy       0.38      0.52      0.44       131
                  late_blight       0.16      0.09      0.12       124
                    leaf_mold       0.64      0.30      0.41       117
           septoria_leaf_spot       0.31      0.53      0.39       114
                  target_spot       0.36      0.30      0.33       111
          tomato_mosaic_virus       0.18      0.27      0.21        60
tomato_yellow_leaf_curl_virus       0.16      0.08      0.11       121
      two-spotted_spider_mite       0.45      0.44      0.44       124

                     accuracy                           0.34      1140
                    macro avg       0.33      0.34      0.32      1140
                 weighted avg       0.34      0.34      0.32      1140

[[8

# Decision Tree on Grayscale

In [66]:
classifier_tree.fit(Ftrain, Ltrain)
classifier_tree.score(Ftest,Ltest)

0.3298245614035088

In [67]:
y_predict = classifier_tree.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.46      0.44      0.45       124
                 early_blight       0.21      0.25      0.22       114
                      healthy       0.47      0.53      0.50       131
                  late_blight       0.38      0.28      0.33       124
                    leaf_mold       0.35      0.32      0.33       117
           septoria_leaf_spot       0.36      0.38      0.37       114
                  target_spot       0.30      0.34      0.32       111
          tomato_mosaic_virus       0.23      0.22      0.22        60
tomato_yellow_leaf_curl_virus       0.23      0.21      0.22       121
      two-spotted_spider_mite       0.31      0.32      0.32       124

                     accuracy                           0.34      1140
                    macro avg       0.33      0.33      0.33      1140
                 weighted avg       0.34      0.34      0.34      1140

[[5

# KNN on Red channel data

In [68]:
#reading numpy array data
red_test = np.load('red_data.npy')
#reading csv file
red_df = pd.read_csv('red_data.csv')

In [69]:
loaded = []
for i in red_test:
    k = []
    k = np.asarray(i,dtype='uint8')
    loaded.append(k)
features = loaded
target = red_df['disease']
Ftrain,Ftest,Ltrain,Ltest=train_test_split(features,target, test_size=0.20, random_state=20)
scaler = StandardScaler()
Ftrain = scaler.fit_transform(Ftrain)
Ftest = scaler.fit_transform(Ftest)

In [70]:
kmodel.fit(Ftrain,Ltrain)
kmodel.score(Ftest,Ltest)

0.4052631578947368

In [71]:
y_predict = kmodel.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.33      0.70      0.45       124
                 early_blight       0.00      0.00      0.00       114
                      healthy       0.36      0.77      0.49       131
                  late_blight       0.34      0.23      0.27       124
                    leaf_mold       0.52      0.46      0.49       117
           septoria_leaf_spot       0.45      0.32      0.38       114
                  target_spot       0.41      0.29      0.34       111
          tomato_mosaic_virus       0.54      0.35      0.42        60
tomato_yellow_leaf_curl_virus       0.40      0.20      0.27       121
      two-spotted_spider_mite       0.53      0.63      0.58       124

                     accuracy                           0.41      1140
                    macro avg       0.39      0.40      0.37      1140
                 weighted avg       0.38      0.41      0.37      1140

[[ 

# Logistic on Red channel data

In [72]:
model.fit(Ftrain,Ltrain)
model.score(Ftest,Ltest)

0.33421052631578946

In [73]:
y_predict = model.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.34      0.40      0.37       124
                 early_blight       0.23      0.25      0.24       114
                      healthy       0.49      0.41      0.45       131
                  late_blight       0.16      0.18      0.17       124
                    leaf_mold       0.40      0.42      0.41       117
           septoria_leaf_spot       0.32      0.32      0.32       114
                  target_spot       0.29      0.25      0.27       111
          tomato_mosaic_virus       0.35      0.22      0.27        60
tomato_yellow_leaf_curl_virus       0.30      0.29      0.29       121
      two-spotted_spider_mite       0.49      0.52      0.50       124

                     accuracy                           0.33      1140
                    macro avg       0.34      0.33      0.33      1140
                 weighted avg       0.34      0.33      0.33      1140

[[5

# SVM(linear) on Red channel data

In [74]:
clf.fit(Ftrain,Ltrain)
y_pred = clf.predict(Ftest)
print(accuracy_score(Ltest,y_pred))

0.3912280701754386


In [75]:
y_predict = clf.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.34      0.46      0.39       124
                 early_blight       0.28      0.32      0.30       114
                      healthy       0.53      0.55      0.54       131
                  late_blight       0.28      0.29      0.29       124
                    leaf_mold       0.45      0.46      0.46       117
           septoria_leaf_spot       0.32      0.32      0.32       114
                  target_spot       0.30      0.26      0.28       111
          tomato_mosaic_virus       0.56      0.33      0.42        60
tomato_yellow_leaf_curl_virus       0.42      0.34      0.37       121
      two-spotted_spider_mite       0.55      0.52      0.53       124

                     accuracy                           0.39      1140
                    macro avg       0.40      0.39      0.39      1140
                 weighted avg       0.40      0.39      0.39      1140

[[5

# SVM(poly) on Red data

In [76]:
svm_poly.fit(Ftrain,Ltrain)
y_pred = svm_poly.predict(Ftest)
print(accuracy_score(Ltest,y_pred))

0.5324561403508772


In [77]:
#classification report
y_predict = svm_poly.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.56      0.59      0.57       124
                 early_blight       0.56      0.16      0.25       114
                      healthy       0.44      0.96      0.61       131
                  late_blight       0.58      0.27      0.37       124
                    leaf_mold       0.65      0.59      0.62       117
           septoria_leaf_spot       0.65      0.46      0.54       114
                  target_spot       0.47      0.68      0.56       111
          tomato_mosaic_virus       0.67      0.13      0.22        60
tomato_yellow_leaf_curl_virus       0.45      0.57      0.50       121
      two-spotted_spider_mite       0.69      0.65      0.67       124

                     accuracy                           0.53      1140
                    macro avg       0.57      0.51      0.49      1140
                 weighted avg       0.57      0.53      0.51      1140

[[ 

# SVM(RBF) on Red data

In [78]:
svm_rbf.fit(Ftrain,Ltrain)
y_pred = svm_rbf.predict(Ftest)
print(accuracy_score(Ltest,y_pred))

0.6403508771929824


In [79]:
#classification report
y_predict = svm_rbf.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.72      0.64      0.68       124
                 early_blight       0.47      0.47      0.47       114
                      healthy       0.74      0.88      0.80       131
                  late_blight       0.60      0.56      0.58       124
                    leaf_mold       0.71      0.66      0.68       117
           septoria_leaf_spot       0.55      0.55      0.55       114
                  target_spot       0.61      0.69      0.65       111
          tomato_mosaic_virus       0.62      0.53      0.57        60
tomato_yellow_leaf_curl_virus       0.66      0.59      0.62       121
      two-spotted_spider_mite       0.68      0.74      0.71       124

                     accuracy                           0.64      1140
                    macro avg       0.64      0.63      0.63      1140
                 weighted avg       0.64      0.64      0.64      1140

[[ 

# SVM(sigmoid) on Red data

In [80]:
svm_si.fit(Ftrain,Ltrain)
y_pred = svm_si.predict(Ftest)
print(accuracy_score(Ltest,y_pred))

0.3


In [81]:
#classification report
y_predict = svm_si.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.39      0.42      0.40       124
                 early_blight       0.15      0.11      0.13       114
                      healthy       0.43      0.27      0.34       131
                  late_blight       0.17      0.35      0.23       124
                    leaf_mold       0.52      0.25      0.34       117
           septoria_leaf_spot       0.20      0.47      0.28       114
                  target_spot       0.38      0.32      0.35       111
          tomato_mosaic_virus       0.38      0.13      0.20        60
tomato_yellow_leaf_curl_virus       0.34      0.19      0.24       121
      two-spotted_spider_mite       0.60      0.40      0.48       124

                     accuracy                           0.30      1140
                    macro avg       0.36      0.29      0.30      1140
                 weighted avg       0.36      0.30      0.30      1140

[[5

# Random forest on Red channel data

In [82]:
classifier_rf.fit(Ftrain, Ltrain)
classifier_rf.oob_score_

0.4678657600350954

In [83]:
y_predict = classifier_rf.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.48      0.61      0.54       124
                 early_blight       0.40      0.15      0.22       114
                      healthy       0.52      0.84      0.64       131
                  late_blight       0.49      0.28      0.36       124
                    leaf_mold       0.56      0.62      0.59       117
           septoria_leaf_spot       0.50      0.46      0.48       114
                  target_spot       0.43      0.70      0.54       111
          tomato_mosaic_virus       0.00      0.00      0.00        60
tomato_yellow_leaf_curl_virus       0.41      0.31      0.36       121
      two-spotted_spider_mite       0.53      0.63      0.57       124

                     accuracy                           0.49      1140
                    macro avg       0.43      0.46      0.43      1140
                 weighted avg       0.46      0.49      0.45      1140

[[ 

C:\Users\ravic\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ravic\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ravic\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Gaussian Naive Bayes on Red channel data

In [84]:
gnb.fit(Ftrain, Ltrain)

GaussianNB()

In [85]:
y_pred = gnb.predict(Ftest)
metrics.accuracy_score(Ltest, y_pred)

0.4342105263157895

In [86]:
class_names = target.unique()
print(classification_report(Ltest, y_pred, target_names=class_names))
print(confusion_matrix(Ltest, y_pred))

                               precision    recall  f1-score   support

               bacterial_spot       0.48      0.69      0.56       124
                 early_blight       0.28      0.24      0.26       114
                      healthy       0.49      0.69      0.58       131
                  late_blight       0.38      0.24      0.29       124
                    leaf_mold       0.59      0.38      0.47       117
           septoria_leaf_spot       0.38      0.41      0.40       114
                  target_spot       0.50      0.43      0.46       111
          tomato_mosaic_virus       0.36      0.58      0.44        60
tomato_yellow_leaf_curl_virus       0.36      0.14      0.20       121
      two-spotted_spider_mite       0.43      0.56      0.49       124

                     accuracy                           0.43      1140
                    macro avg       0.43      0.44      0.42      1140
                 weighted avg       0.43      0.43      0.42      1140

[[8

# Bernouli Naive bayes on Red data

In [87]:
bnb.fit(Ftrain, Ltrain)
y_pred = bnb.predict(Ftest)
metrics.accuracy_score(Ltest, y_pred)

0.3526315789473684

In [88]:
class_names = target.unique()
print(classification_report(Ltest, y_pred, target_names=class_names))
print(confusion_matrix(Ltest, y_pred))

                               precision    recall  f1-score   support

               bacterial_spot       0.39      0.70      0.50       124
                 early_blight       0.19      0.11      0.14       114
                      healthy       0.41      0.54      0.47       131
                  late_blight       0.21      0.10      0.14       124
                    leaf_mold       0.51      0.16      0.25       117
           septoria_leaf_spot       0.28      0.48      0.36       114
                  target_spot       0.42      0.41      0.41       111
          tomato_mosaic_virus       0.32      0.32      0.32        60
tomato_yellow_leaf_curl_virus       0.27      0.13      0.18       121
      two-spotted_spider_mite       0.40      0.52      0.45       124

                     accuracy                           0.35      1140
                    macro avg       0.34      0.35      0.32      1140
                 weighted avg       0.34      0.35      0.32      1140

[[8

# Decision Tree on Red channel

In [89]:
classifier_tree.fit(Ftrain, Ltrain)
classifier_tree.score(Ftest,Ltest)

0.36140350877192984

In [90]:
y_predict = classifier_tree.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.52      0.52      0.52       124
                 early_blight       0.26      0.23      0.24       114
                      healthy       0.55      0.53      0.54       131
                  late_blight       0.37      0.39      0.38       124
                    leaf_mold       0.40      0.40      0.40       117
           septoria_leaf_spot       0.39      0.41      0.40       114
                  target_spot       0.26      0.30      0.28       111
          tomato_mosaic_virus       0.17      0.17      0.17        60
tomato_yellow_leaf_curl_virus       0.25      0.25      0.25       121
      two-spotted_spider_mite       0.35      0.32      0.34       124

                     accuracy                           0.36      1140
                    macro avg       0.35      0.35      0.35      1140
                 weighted avg       0.37      0.36      0.36      1140

[[6

# KNN on green channel data

In [91]:
#reading numpy array data
green_test = np.load('green_data.npy')
#reading csv file
green_df = pd.read_csv('green_data.csv')
loaded = []
for i in green_test:
    k = []
    k = np.asarray(i,dtype='uint8')
    loaded.append(k)
features = loaded
target = green_df['disease']
Ftrain,Ftest,Ltrain,Ltest=train_test_split(features,target, test_size=0.20, random_state=20)
scaler = StandardScaler()
Ftrain = scaler.fit_transform(Ftrain)
Ftest = scaler.fit_transform(Ftest)
kmodel.fit(Ftrain,Ltrain)
kmodel.score(Ftest,Ltest)

0.3280701754385965

In [92]:
y_predict = kmodel.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.23      0.74      0.36       124
                 early_blight       0.00      0.00      0.00       114
                      healthy       0.33      0.60      0.43       131
                  late_blight       0.24      0.18      0.21       124
                    leaf_mold       0.41      0.51      0.45       117
           septoria_leaf_spot       0.45      0.39      0.42       114
                  target_spot       0.34      0.09      0.14       111
          tomato_mosaic_virus       0.33      0.08      0.13        60
tomato_yellow_leaf_curl_virus       0.59      0.11      0.18       121
      two-spotted_spider_mite       0.46      0.40      0.42       124

                     accuracy                           0.33      1140
                    macro avg       0.34      0.31      0.27      1140
                 weighted avg       0.34      0.33      0.29      1140

[[9

# Logistic on Green channel data

In [93]:
model.fit(Ftrain,Ltrain)
model.score(Ftest,Ltest)

0.3131578947368421

In [94]:
y_predict = model.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.34      0.40      0.36       124
                 early_blight       0.25      0.25      0.25       114
                      healthy       0.48      0.44      0.46       131
                  late_blight       0.19      0.23      0.21       124
                    leaf_mold       0.35      0.39      0.37       117
           septoria_leaf_spot       0.31      0.32      0.32       114
                  target_spot       0.20      0.17      0.18       111
          tomato_mosaic_virus       0.14      0.07      0.09        60
tomato_yellow_leaf_curl_virus       0.32      0.29      0.30       121
      two-spotted_spider_mite       0.41      0.42      0.41       124

                     accuracy                           0.31      1140
                    macro avg       0.30      0.30      0.30      1140
                 weighted avg       0.31      0.31      0.31      1140

[[4

# SVM(linear) on green channel data

In [95]:
clf.fit(Ftrain,Ltrain)
y_pred = clf.predict(Ftest)
print(accuracy_score(Ltest,y_pred))

0.37719298245614036


In [96]:
y_predict = clf.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.32      0.45      0.38       124
                 early_blight       0.30      0.35      0.32       114
                      healthy       0.53      0.53      0.53       131
                  late_blight       0.29      0.32      0.31       124
                    leaf_mold       0.41      0.43      0.42       117
           septoria_leaf_spot       0.40      0.39      0.39       114
                  target_spot       0.26      0.23      0.25       111
          tomato_mosaic_virus       0.44      0.18      0.26        60
tomato_yellow_leaf_curl_virus       0.38      0.33      0.35       121
      two-spotted_spider_mite       0.50      0.44      0.47       124

                     accuracy                           0.38      1140
                    macro avg       0.39      0.36      0.37      1140
                 weighted avg       0.38      0.38      0.38      1140

[[5

# SVM(poly) on Green data

In [97]:
svm_poly.fit(Ftrain,Ltrain)
y_pred = svm_poly.predict(Ftest)
print(accuracy_score(Ltest,y_pred))

0.4666666666666667


In [98]:
#classification report
y_predict = svm_poly.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.37      0.76      0.50       124
                 early_blight       0.57      0.23      0.32       114
                      healthy       0.49      0.70      0.58       131
                  late_blight       0.60      0.10      0.17       124
                    leaf_mold       0.33      0.70      0.45       117
           septoria_leaf_spot       0.65      0.46      0.54       114
                  target_spot       0.49      0.49      0.49       111
          tomato_mosaic_virus       0.20      0.02      0.03        60
tomato_yellow_leaf_curl_virus       0.58      0.40      0.48       121
      two-spotted_spider_mite       0.68      0.56      0.61       124

                     accuracy                           0.47      1140
                    macro avg       0.50      0.44      0.42      1140
                 weighted avg       0.51      0.47      0.44      1140

[[9

# SVM(RBF) on green data

In [99]:
svm_rbf.fit(Ftrain,Ltrain)
y_pred = svm_rbf.predict(Ftest)
print(accuracy_score(Ltest,y_pred))

0.6228070175438597


In [100]:
#classification report
y_predict = svm_rbf.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.75      0.69      0.72       124
                 early_blight       0.44      0.48      0.46       114
                      healthy       0.69      0.84      0.76       131
                  late_blight       0.61      0.53      0.57       124
                    leaf_mold       0.65      0.62      0.63       117
           septoria_leaf_spot       0.57      0.66      0.61       114
                  target_spot       0.54      0.61      0.57       111
          tomato_mosaic_virus       0.53      0.40      0.46        60
tomato_yellow_leaf_curl_virus       0.66      0.55      0.60       121
      two-spotted_spider_mite       0.74      0.69      0.72       124

                     accuracy                           0.62      1140
                    macro avg       0.62      0.61      0.61      1140
                 weighted avg       0.63      0.62      0.62      1140

[[ 

# SVM(sigmoid) on green data

In [101]:
svm_si.fit(Ftrain,Ltrain)
y_pred = svm_si.predict(Ftest)
print(accuracy_score(Ltest,y_pred))

0.2543859649122807


In [102]:
#classification report
y_predict = svm_si.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.39      0.45      0.42       124
                 early_blight       0.20      0.17      0.18       114
                      healthy       0.42      0.26      0.32       131
                  late_blight       0.15      0.29      0.20       124
                    leaf_mold       0.57      0.17      0.26       117
           septoria_leaf_spot       0.17      0.45      0.25       114
                  target_spot       0.20      0.20      0.20       111
          tomato_mosaic_virus       0.00      0.00      0.00        60
tomato_yellow_leaf_curl_virus       0.27      0.13      0.18       121
      two-spotted_spider_mite       0.40      0.29      0.34       124

                     accuracy                           0.25      1140
                    macro avg       0.28      0.24      0.24      1140
                 weighted avg       0.30      0.25      0.25      1140

[[5

# Random Forest on green channel data

In [103]:
classifier_rf.fit(Ftrain, Ltrain)
classifier_rf.oob_score_

0.42399649045843385

In [104]:
y_predict = classifier_rf.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.42      0.60      0.49       124
                 early_blight       0.36      0.24      0.29       114
                      healthy       0.43      0.68      0.53       131
                  late_blight       0.46      0.29      0.35       124
                    leaf_mold       0.48      0.50      0.49       117
           septoria_leaf_spot       0.57      0.49      0.53       114
                  target_spot       0.33      0.58      0.42       111
          tomato_mosaic_virus       0.00      0.00      0.00        60
tomato_yellow_leaf_curl_virus       0.52      0.25      0.34       121
      two-spotted_spider_mite       0.45      0.47      0.46       124

                     accuracy                           0.43      1140
                    macro avg       0.40      0.41      0.39      1140
                 weighted avg       0.42      0.43      0.41      1140

[[7

C:\Users\ravic\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ravic\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ravic\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Gaussian Naive bayes on Green channel data

In [105]:
gnb.fit(Ftrain, Ltrain)
y_pred = gnb.predict(Ftest)
metrics.accuracy_score(Ltest, y_pred)

0.3850877192982456

In [106]:
class_names = target.unique()
print(classification_report(Ltest, y_pred, target_names=class_names))
print(confusion_matrix(Ltest, y_pred))

                               precision    recall  f1-score   support

               bacterial_spot       0.47      0.68      0.55       124
                 early_blight       0.38      0.39      0.38       114
                      healthy       0.43      0.61      0.50       131
                  late_blight       0.31      0.19      0.24       124
                    leaf_mold       0.52      0.21      0.30       117
           septoria_leaf_spot       0.36      0.45      0.40       114
                  target_spot       0.49      0.35      0.41       111
          tomato_mosaic_virus       0.26      0.53      0.35        60
tomato_yellow_leaf_curl_virus       0.20      0.09      0.13       121
      two-spotted_spider_mite       0.38      0.39      0.38       124

                     accuracy                           0.39      1140
                    macro avg       0.38      0.39      0.36      1140
                 weighted avg       0.38      0.39      0.37      1140

[[8

# Bernouli Naive bayes on green data

In [107]:
bnb.fit(Ftrain, Ltrain)
y_pred = bnb.predict(Ftest)
metrics.accuracy_score(Ltest, y_pred)

0.32456140350877194

In [108]:
class_names = target.unique()
print(classification_report(Ltest, y_pred, target_names=class_names))
print(confusion_matrix(Ltest, y_pred))

                               precision    recall  f1-score   support

               bacterial_spot       0.40      0.68      0.50       124
                 early_blight       0.24      0.16      0.19       114
                      healthy       0.40      0.54      0.46       131
                  late_blight       0.27      0.14      0.18       124
                    leaf_mold       0.61      0.20      0.30       117
           septoria_leaf_spot       0.32      0.59      0.41       114
                  target_spot       0.33      0.21      0.26       111
          tomato_mosaic_virus       0.12      0.30      0.18        60
tomato_yellow_leaf_curl_virus       0.06      0.02      0.03       121
      two-spotted_spider_mite       0.44      0.37      0.40       124

                     accuracy                           0.32      1140
                    macro avg       0.32      0.32      0.29      1140
                 weighted avg       0.33      0.32      0.30      1140

[[8

# Decision Tree on Green channel

In [109]:
classifier_tree.fit(Ftrain, Ltrain)
classifier_tree.score(Ftest,Ltest)

0.3219298245614035

In [110]:
y_predict = classifier_tree.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.36      0.35      0.35       124
                 early_blight       0.20      0.21      0.20       114
                      healthy       0.49      0.44      0.46       131
                  late_blight       0.38      0.35      0.36       124
                    leaf_mold       0.32      0.37      0.34       117
           septoria_leaf_spot       0.33      0.43      0.38       114
                  target_spot       0.24      0.23      0.23       111
          tomato_mosaic_virus       0.13      0.13      0.13        60
tomato_yellow_leaf_curl_virus       0.32      0.31      0.31       121
      two-spotted_spider_mite       0.34      0.29      0.31       124

                     accuracy                           0.32      1140
                    macro avg       0.31      0.31      0.31      1140
                 weighted avg       0.32      0.32      0.32      1140

[[4

# KNN on blue channel data

In [111]:
#reading numpy array data
blue_test = np.load('blue_data.npy')
#reading csv file
blue_df = pd.read_csv('blue_data.csv')
loaded = []
for i in gray_test:
    k = []
    k = np.asarray(i,dtype='uint8')
    loaded.append(k)
features = loaded
target = blue_df['disease']
Ftrain,Ftest,Ltrain,Ltest=train_test_split(features,target, test_size=0.20, random_state=20)
Ftrain = scaler.fit_transform(Ftrain)
Ftest = scaler.fit_transform(Ftest)
kmodel.fit(Ftrain,Ltrain)
kmodel.score(Ftest,Ltest)

0.356140350877193

In [112]:
y_predict = kmodel.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.27      0.74      0.39       124
                 early_blight       0.00      0.00      0.00       114
                      healthy       0.36      0.69      0.47       131
                  late_blight       0.26      0.23      0.24       124
                    leaf_mold       0.48      0.50      0.49       117
           septoria_leaf_spot       0.44      0.42      0.43       114
                  target_spot       0.38      0.15      0.22       111
          tomato_mosaic_virus       0.26      0.12      0.16        60
tomato_yellow_leaf_curl_virus       0.45      0.12      0.18       121
      two-spotted_spider_mite       0.50      0.42      0.46       124

                     accuracy                           0.36      1140
                    macro avg       0.34      0.34      0.30      1140
                 weighted avg       0.34      0.36      0.32      1140

[[9

C:\Users\ravic\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ravic\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ravic\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# logistic on blue channel data

In [113]:
model.fit(Ftrain,Ltrain)
model.score(Ftest,Ltest)

0.3157894736842105

In [114]:
y_predict = model.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.32      0.38      0.35       124
                 early_blight       0.24      0.28      0.26       114
                      healthy       0.49      0.44      0.46       131
                  late_blight       0.18      0.21      0.20       124
                    leaf_mold       0.38      0.38      0.38       117
           septoria_leaf_spot       0.30      0.30      0.30       114
                  target_spot       0.21      0.18      0.19       111
          tomato_mosaic_virus       0.28      0.13      0.18        60
tomato_yellow_leaf_curl_virus       0.27      0.25      0.26       121
      two-spotted_spider_mite       0.45      0.49      0.47       124

                     accuracy                           0.32      1140
                    macro avg       0.31      0.30      0.30      1140
                 weighted avg       0.32      0.32      0.31      1140

[[4

# SVM(linear) on blue channel data

In [115]:
clf.fit(Ftrain,Ltrain)
y_pred = clf.predict(Ftest)
print(accuracy_score(Ltest,y_pred))

0.3868421052631579


In [116]:
y_predict = clf.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.33      0.44      0.38       124
                 early_blight       0.29      0.35      0.32       114
                      healthy       0.53      0.53      0.53       131
                  late_blight       0.31      0.32      0.32       124
                    leaf_mold       0.44      0.46      0.45       117
           septoria_leaf_spot       0.39      0.35      0.37       114
                  target_spot       0.29      0.26      0.27       111
          tomato_mosaic_virus       0.41      0.23      0.30        60
tomato_yellow_leaf_curl_virus       0.38      0.32      0.35       121
      two-spotted_spider_mite       0.52      0.49      0.50       124

                     accuracy                           0.39      1140
                    macro avg       0.39      0.38      0.38      1140
                 weighted avg       0.39      0.39      0.39      1140

[[5

# SVM(poly) on blue data

In [117]:
svm_poly.fit(Ftrain,Ltrain)
y_pred = svm_poly.predict(Ftest)
print(accuracy_score(Ltest,y_pred))

0.5026315789473684


In [118]:
#classification report
y_predict = svm_poly.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.44      0.68      0.53       124
                 early_blight       0.54      0.19      0.28       114
                      healthy       0.48      0.86      0.62       131
                  late_blight       0.50      0.15      0.23       124
                    leaf_mold       0.50      0.65      0.56       117
           septoria_leaf_spot       0.68      0.42      0.52       114
                  target_spot       0.47      0.56      0.51       111
          tomato_mosaic_virus       0.60      0.05      0.09        60
tomato_yellow_leaf_curl_virus       0.42      0.57      0.48       121
      two-spotted_spider_mite       0.69      0.62      0.65       124

                     accuracy                           0.50      1140
                    macro avg       0.53      0.48      0.45      1140
                 weighted avg       0.53      0.50      0.47      1140

[[ 

# SVM(RBF) on blue data

In [119]:
svm_rbf.fit(Ftrain,Ltrain)
y_pred = svm_rbf.predict(Ftest)
print(accuracy_score(Ltest,y_pred))

0.6280701754385964


In [120]:
#classification report
y_predict = svm_rbf.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.75      0.65      0.70       124
                 early_blight       0.46      0.49      0.48       114
                      healthy       0.70      0.87      0.77       131
                  late_blight       0.61      0.54      0.58       124
                    leaf_mold       0.67      0.62      0.64       117
           septoria_leaf_spot       0.56      0.61      0.58       114
                  target_spot       0.55      0.63      0.59       111
          tomato_mosaic_virus       0.60      0.47      0.52        60
tomato_yellow_leaf_curl_virus       0.64      0.61      0.63       121
      two-spotted_spider_mite       0.72      0.69      0.70       124

                     accuracy                           0.63      1140
                    macro avg       0.63      0.62      0.62      1140
                 weighted avg       0.63      0.63      0.63      1140

[[ 

# SVM(sigmoid) on blue data

In [121]:
svm_si.fit(Ftrain,Ltrain)
y_pred = svm_si.predict(Ftest)
print(accuracy_score(Ltest,y_pred))

0.27807017543859647


In [122]:
#classification report
y_predict = svm_si.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.38      0.47      0.42       124
                 early_blight       0.18      0.20      0.19       114
                      healthy       0.35      0.22      0.27       131
                  late_blight       0.19      0.27      0.22       124
                    leaf_mold       0.57      0.22      0.32       117
           septoria_leaf_spot       0.17      0.37      0.23       114
                  target_spot       0.21      0.23      0.22       111
          tomato_mosaic_virus       0.46      0.10      0.16        60
tomato_yellow_leaf_curl_virus       0.34      0.23      0.27       121
      two-spotted_spider_mite       0.47      0.38      0.42       124

                     accuracy                           0.28      1140
                    macro avg       0.33      0.27      0.27      1140
                 weighted avg       0.33      0.28      0.28      1140

[[5

# Random Forest on blue channel data

In [123]:
classifier_rf.fit(Ftrain, Ltrain)
classifier_rf.oob_score_

0.442640930028515

In [124]:
y_predict = classifier_rf.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.46      0.61      0.53       124
                 early_blight       0.40      0.18      0.25       114
                      healthy       0.47      0.74      0.57       131
                  late_blight       0.48      0.28      0.36       124
                    leaf_mold       0.57      0.54      0.55       117
           septoria_leaf_spot       0.58      0.46      0.51       114
                  target_spot       0.38      0.67      0.48       111
          tomato_mosaic_virus       0.00      0.00      0.00        60
tomato_yellow_leaf_curl_virus       0.43      0.31      0.36       121
      two-spotted_spider_mite       0.48      0.60      0.54       124

                     accuracy                           0.47      1140
                    macro avg       0.42      0.44      0.42      1140
                 weighted avg       0.45      0.47      0.44      1140

[[7

C:\Users\ravic\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ravic\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ravic\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Gaussian Naive bayes on blue channel data

In [125]:
gnb.fit(Ftrain, Ltrain)
y_pred = gnb.predict(Ftest)
metrics.accuracy_score(Ltest, y_pred)

0.41578947368421054

In [126]:
class_names = target.unique()
print(classification_report(Ltest, y_pred, target_names=class_names))
print(confusion_matrix(Ltest, y_pred))

                               precision    recall  f1-score   support

               bacterial_spot       0.49      0.69      0.57       124
                 early_blight       0.38      0.37      0.37       114
                      healthy       0.45      0.63      0.52       131
                  late_blight       0.33      0.20      0.25       124
                    leaf_mold       0.61      0.30      0.40       117
           septoria_leaf_spot       0.40      0.47      0.43       114
                  target_spot       0.51      0.39      0.44       111
          tomato_mosaic_virus       0.29      0.60      0.39        60
tomato_yellow_leaf_curl_virus       0.30      0.13      0.18       121
      two-spotted_spider_mite       0.40      0.44      0.42       124

                     accuracy                           0.42      1140
                    macro avg       0.41      0.42      0.40      1140
                 weighted avg       0.42      0.42      0.40      1140

[[8

# Bernouli Naive bayes on blue data

In [127]:
bnb.fit(Ftrain, Ltrain)
y_pred = bnb.predict(Ftest)
metrics.accuracy_score(Ltest, y_pred)

0.34385964912280703

In [128]:
class_names = target.unique()
print(classification_report(Ltest, y_pred, target_names=class_names))
print(confusion_matrix(Ltest, y_pred))

                               precision    recall  f1-score   support

               bacterial_spot       0.40      0.69      0.51       124
                 early_blight       0.27      0.18      0.21       114
                      healthy       0.38      0.52      0.44       131
                  late_blight       0.16      0.09      0.12       124
                    leaf_mold       0.64      0.30      0.41       117
           septoria_leaf_spot       0.31      0.53      0.39       114
                  target_spot       0.36      0.30      0.33       111
          tomato_mosaic_virus       0.18      0.27      0.21        60
tomato_yellow_leaf_curl_virus       0.16      0.08      0.11       121
      two-spotted_spider_mite       0.45      0.44      0.44       124

                     accuracy                           0.34      1140
                    macro avg       0.33      0.34      0.32      1140
                 weighted avg       0.34      0.34      0.32      1140

[[8

# Decision Tree on Blue channel

In [129]:
classifier_tree.fit(Ftrain, Ltrain)
classifier_tree.score(Ftest,Ltest)

0.3508771929824561

In [130]:
y_predict = classifier_tree.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.46      0.44      0.45       124
                 early_blight       0.22      0.24      0.23       114
                      healthy       0.49      0.53      0.51       131
                  late_blight       0.34      0.27      0.30       124
                    leaf_mold       0.38      0.33      0.35       117
           septoria_leaf_spot       0.35      0.39      0.37       114
                  target_spot       0.31      0.37      0.34       111
          tomato_mosaic_virus       0.18      0.18      0.18        60
tomato_yellow_leaf_curl_virus       0.28      0.26      0.27       121
      two-spotted_spider_mite       0.36      0.36      0.36       124

                     accuracy                           0.35      1140
                    macro avg       0.34      0.34      0.34      1140
                 weighted avg       0.35      0.35      0.35      1140

[[5

In [131]:
end_time = time.time()
print(end_time - start_time)

3749.363508462906
